In [1]:
from src.cloud_functions._1_fetch_data._1_13_fetch_items_promotions.main import fetch_promotions_data


In [2]:
class MockRequest:
    
    def __init__(self, json_data):
        self._json_data = json_data

    def get_json(self):
        return self._json_data

In [3]:
test_data = {
 "access_token": None,
 "client_id": "4959083987776428",
 "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
 "seller_id": 189643563,
 "store_name": "hubsmarthome"
}

mock_request = MockRequest(test_data)

In [ ]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime
import time

semaphore = asyncio.Semaphore(100)  # Control the number of simultaneous requests

async def main_async(request):
    # Parsing request data
    data = request.get_json()
    client_id = data.get('client_id')
    client_secret = data.get('client_secret')
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')
    access_token = data.get('access_token')
    print('** Defining authentication... **')
    
    # Authenticate (assuming this is now centralized in utils.py or a similar file)
    if not access_token:
        access_token = authenticate(client_id, client_secret)  # You can add this to a common module
    print('** Connecting to storage and BigQuery... **')

    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    #bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    
    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_ITEM_PROMOTION
    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')
    
    # Fetch item IDs from the storage bucket
    blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
    items_id = fetch_items_from_storage(storage, settings.BUCKET_STORES, blob_items_prefix, 'id')
    print(f'** Items found: {len(items_id)}**')
    
    print(f'** Cleaning blob **')
    # Path for saving promotions details
    # marketplace
    blob_basic_path_marketplace = settings.BLOB_PROMOTIONS(store_name)
    date_blob_path_marketplace = f'{blob_basic_path_marketplace}date={today_str}/'
    
    # mshops
    blob_basic_path_mshops = settings.BLOB_PROMOTIONS_MSHOPS(store_name)
    date_blob_path_mshops = f'{blob_basic_path_mshops}date={today_str}/'
    
    # Clean existing files in the storage bucket
    storage.clean_blobs(bucket_name, date_blob_path_marketplace)
    storage.clean_blobs(bucket_name, date_blob_path_mshops)
    print(f'** Starting API requests for {len(items_id)} items**')
    # URL function for API
    url_marketplace = settings.URL_PROMOTIONS_MARKETPLACE
    url_mshops = settings.URL_PROMOTIONS_MSHOPS
    headers = {'Authorization': f'Bearer {access_token}'}
    
    # Batch processing the API requests
    # PROMOTIONS MELI
    async with aiohttp.ClientSession() as session:
        await batch_process(session, items_id, url_marketplace, headers, bucket_name, date_blob_path_marketplace, storage, add_item_id = True, sleep_time=5)
        
    # Batch processing the API requests
    # PROMOTIONS SHOPS
    async with aiohttp.ClientSession() as session:
        await batch_process(session, items_id, url_mshops, headers, bucket_name, date_blob_path_mshops, storage, add_item_id = True, sleep_time=5)


    
    print('** Logging process in management table... **')
    # Log the process in BigQuery
    log_process(seller_id, destiny_table, today_str, table_management, processed_to_bq=False)

    return ('Success', 200)

def fetch_promotions_data(request):
    return (main_async(request))



In [5]:
result = await fetch_promotions_data(mock_request)


** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: D:/bacar/Savvi/GLM/Desenvolvimento/service_account/service_account_datalakev2.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-06/batch_0__process_time=2024-11-06T07:00:00.681709-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-06/batch_10__process_time=2024-11-06T07:00:00.223338-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-06/batch_1__process_time=2024-11-06T07:00:00.949312-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-06/batch_2__process_time=2024-11-06T07:00:00.289432-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-06/batch_3__process_time=2024-11-06T07:00:00.658403-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-11-06/batch_4__process_time=2024-11-06T07:00:00.924985-03:00.json
Reading fil

In [ ]:
import pandas as pd
from datetime import datetime
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings

def insert_bq_promotions(request):
    data = request.get_json()
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')

    print('** Connecting to storage and BigQuery... **')
    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_ITEM_PROMOTION
    blob_promotions = settings.BLOB_PROMOTIONS(store_name)
    blob_promotions_mshops = settings.BLOB_PROMOTIONS_MSHOPS(store_name)

    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')

    # Get dates to treat
    list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)

    print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process ***')

    for date in list_dates_to_process:
        date_to_process = date.strftime('%Y-%m-%d')
        print(f'Processing date: {date_to_process}')

        processed_data = []

        # Channel 'marketplace' processing
        blob_prefix = blob_promotions + f'date={date_to_process}/'
        blobs = storage.list_blobs(bucket_name, blob_prefix)
        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = storage.download_json(bucket_name, blob.name)
            for json_item in content:
                if json_item:  # Ensure json_item is not None or empty
                    for item in json_item:
                        if item:
                            processed_dict = process_response(item, 'Marketplace')
                            if processed_dict:
                                processed_data.append(processed_dict)

        # Channel 'mshops' processing
        blob_prefix_mshops = blob_promotions_mshops + f'date={date_to_process}/'
        blobs = storage.list_blobs(bucket_name, blob_prefix_mshops)
        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = storage.download_json(bucket_name, blob.name)
            for json_item in content:
                if json_item:  # Ensure json_item is not None or empty
                    for item in json_item:
                        if item:
                            processed_dict = process_response(item, 'mshops')
                            if processed_dict:
                                processed_data.append(processed_dict)

        df_processed_data = pd.DataFrame(processed_data)

        if df_processed_data.empty:
            print(f'Nenhum dado processado para a data {date_to_process}, pulando inserção...')
            continue

        df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
        df_processed_data['process_time'] = datetime.now()
        df_processed_data['seller_id'] = seller_id

        print(f'*** Finished treating all data. {df_processed_data.shape[0]} products ***')

        if not bigquery.table_exists(destiny_table):
            print(f'Table {destiny_table} does not exist. Creating table...')
            bigquery.create_table(destiny_table, df_processed_data)

        print('** Deleting existing data **')
        bigquery.delete_existing_data(destiny_table, seller_id, date_to_process)

        print('** Correcting dataframe schema **')
        df_processed_data = bigquery.match_dataframe_schema(df_processed_data, destiny_table)

        print('** Inserting data into BQ **')
        bigquery.insert_dataframe(df_processed_data, destiny_table)

        print('** Updating log table **')
        bigquery.update_logs_table(seller_id, date_to_process, destiny_table, table_management)

    return ('Success', 200)


def process_response(json_item, channel):
    # Define required fields for both channels
    required_fields_marketplace = ['item_id', 'id', 'status', 'type', 'name', 'start_date', 'finish_date']
    required_fields_mshops = ['item_id', 'id', 'status', 'type', 'name', 'target', 'buy_quantity', 'start_date', 'finish_date']

    required_fields = required_fields_marketplace if channel == "Marketplace" else required_fields_mshops

    # Check if all required fields are present in the json_item
    for field in required_fields:
        if field not in json_item or json_item[field] is None:
            # Check within "offers" list for fields like "start_date"
            if field == "start_date" and "offers" in json_item:
                start_date_found = any(offer.get("start_date") for offer in json_item["offers"])
                if not start_date_found:
                    print(f'Missing or None field "{field}" in item: {json_item}')

    # Process data based on channel type
    if channel == "Marketplace":
        data = {
            'item_id': json_item['item_id'],
            'promotion_id': json_item['id'],
            'status': json_item['status'],
            'type': json_item['type'],
            'name': json_item['name'],
            'meli_percent': json_item.get('benefits', {}).get('meli_percent'),
            'seller_percent': json_item.get('benefits', {}).get('seller_percent'),
            'start_date': json_item['start_date'] if 'start_date' in json_item else json_item['offers'][0].get('start_date'),
            'finish_date': json_item['finish_date'],
            'channel': channel,
        }

    elif channel == "mshops":
        data = {
            'item_id': json_item['item_id'],
            'promotion_id': json_item['id'],
            'status': json_item['status'],
            'type': json_item['type'],
            'name': json_item['name'],
            'target': json_item['target'],
            'buy_quantity': json_item['buy_quantity'],
            'start_date': json_item['start_date'] if 'start_date' in json_item else json_item['offers'][0].get('start_date'),
            'finish_date': json_item['finish_date'],
            'channel': channel,
        }

    return data  # Return the processed data dictionary


In [ ]:
result = insert_bq_promotions(mock_request)

In [ ]:
import pandas as pd
from datetime import datetime
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings

def insert_bq_promotions(request):
    data = request.get_json()
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')

    print('** Connecting to storage and BigQuery... **')
    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_EXTERNAL_ORDERS
    blob_promotions = settings.BLOB_PROMOTIONS(store_name)
    blob_promotions_mshops = settings.BLOB_PROMOTIONS_MSHOPS(store_name)

    today_str = datetime.today().strftime('%Y-%m-%d')

    
    # Caminho para o arquivo .xlsx
    file_path = 'vendas_ml.xlsx'

    # Lendo o arquivo e criando o DataFrame
    df = pd.read_excel(file_path)

    # Exibindo as primeiras linhas do DataFrame
    print(df.head())

    df_processed_data = df

    if not bigquery.table_exists(destiny_table):
        print(f'Table {destiny_table} does not exist. Creating table...')
        bigquery.create_table(destiny_table, df_processed_data)

    print('** Deleting existing data **')
    bigquery.delete_existing_data(destiny_table, seller_id, today_str)

    print('** Correcting dataframe schema **')
    df_processed_data = bigquery.match_dataframe_schema(df_processed_data, destiny_table)

    print('** Inserting data into BQ **')
    bigquery.insert_dataframe(df_processed_data, destiny_table)

    print('** Updating log table **')
    bigquery.update_logs_table(seller_id, today_str, destiny_table, table_management)

    return ('Success', 200)


def process_response(json_item, channel):
    # Define required fields for both channels
    required_fields_marketplace = ['item_id', 'id', 'status', 'type', 'name', 'start_date', 'finish_date']
    required_fields_mshops = ['item_id', 'id', 'status', 'type', 'name', 'target', 'buy_quantity', 'start_date', 'finish_date']

    required_fields = required_fields_marketplace if channel == "Marketplace" else required_fields_mshops

    # Check if all required fields are present in the json_item
    for field in required_fields:
        if field not in json_item or json_item[field] is None:
            # Check within "offers" list for fields like "start_date"
            if field == "start_date" and "offers" in json_item:
                start_date_found = any(offer.get("start_date") for offer in json_item["offers"])
                if not start_date_found:
                    print(f'Missing or None field "{field}" in item: {json_item}')

    # Process data based on channel type
    if channel == "Marketplace":
        data = {
            'item_id': json_item['item_id'],
            'promotion_id': json_item['id'],
            'status': json_item['status'],
            'type': json_item['type'],
            'name': json_item['name'],
            'meli_percent': json_item.get('benefits', {}).get('meli_percent'),
            'seller_percent': json_item.get('benefits', {}).get('seller_percent'),
            'start_date': json_item['start_date'] if 'start_date' in json_item else json_item['offers'][0].get('start_date'),
            'finish_date': json_item['finish_date'],
            'channel': channel,
        }

    elif channel == "mshops":
        data = {
            'item_id': json_item['item_id'],
            'promotion_id': json_item['id'],
            'status': json_item['status'],
            'type': json_item['type'],
            'name': json_item['name'],
            'target': json_item['target'],
            'buy_quantity': json_item['buy_quantity'],
            'start_date': json_item['start_date'] if 'start_date' in json_item else json_item['offers'][0].get('start_date'),
            'finish_date': json_item['finish_date'],
            'channel': channel,
        }

    return data  # Return the processed data dictionary


In [ ]:
import os
from src.config.settings import *

path_credentials = PATH_SERVICE_ACCOUNT

if os.path.exists(path_credentials):
    print("O arquivo existe.")
else:
    print("O arquivo não foi encontrado.")


O arquivo existe.


In [5]:

bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

  # Get salles table from BigQuery
df_sales = get_sales_table(settings.TABLE_ORDERS, bigquery)
# Create max 7 and 21 days transactions per SKU and channel
df_max_sales = create_max_7_and_21_days_history_per_sku(df_sales)
# Create max 7 and 21 days transactions per SKU and channel
df_max_sales_global = create_max_7_and_21_days_history_per_sku_global(df_sales)
# Get past 7 days sales
df_past_7_days_sales = create_past_sales(df_sales, 7)

# Get past 21 days sales
df_past_21_days_sales = create_past_sales(df_sales, 21)
# Merge tables
df_consolidado = df_max_sales.merge(df_past_7_days_sales, on = ['seller_sku', 'seller_id', 'context_channel'], how = 'left').merge(df_past_21_days_sales, on = ['seller_sku', 'seller_id', 'context_channel'], how = 'left').merge(df_max_sales_global, on = ['seller_sku', 'seller_id'], how = 'left')
# BigQuery dataset and table ID
dataset_table_id = settings.TABLE_PREDICTED_SALES

Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


KeyboardInterrupt: 

NameError: name 'df_consolidado' is not defined

In [33]:
import logging
from datetime import datetime, timedelta
from src.common.bigquery_connector import BigQueryManager
import pandas as pd
import pytz
from src.config import settings

# Configure logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter("%(asctime)s %(name)s [%(levelname)s] %(message)s"))
logger.addHandler(handler)

def get_max_sales_history(request):
    logger.info("Starting the get_max_sales_history process")

    # Initialize BigQuery manager
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    logger.info("BigQueryManager initialized")

    # Get sales data
    df_sales = get_sales_table(settings.TABLE_ORDERS, bigquery)
    logger.info(f"Loaded {len(df_sales)} sales records from BigQuery")

    # Compute rolling maxima per SKU + channel and globally
    logger.info("Computing maximum sales history per SKU and channel")
    df_max_sales = create_max_history_per_sku(df_sales, context_channel=True)
    logger.info("7 and 21 days max sales per SKU and channel computed")

    logger.info("Computing maximum sales history per SKU (global)")
    df_max_sales_global = create_max_history_per_sku(df_sales, context_channel=False)
    logger.info("7 and 21 days max sales per SKU (global) computed")

    # Compute past sales
    logger.info("Computing past 7 days sales")
    df_past_7_days_sales_with_channel, df_past_7_days_sales = create_past_sales(df_sales, 7)

    logger.info("Computing past 21 days sales")
    df_past_21_days_sales_with_channel, df_past_21_days_sales = create_past_sales(df_sales, 21)

    # Merge results
    logger.info("Merging all computed dataframes")
    # For global max, we do not have context_channel, so merge on just sku and seller_id first.
    df_consolidado = (
        df_max_sales
        .merge(df_past_21_days_sales_with_channel, on=['seller_sku', 'seller_id', 'context_channel'], how='left')
        .merge(df_past_7_days_sales_with_channel, on=['seller_sku', 'seller_id', 'context_channel'], how='left')
        .merge(df_max_sales_global, on=['seller_sku', 'seller_id'], how='left', suffixes=('', '_global'))
        .merge(df_past_21_days_sales, on=['seller_sku', 'seller_id'], how='left', suffixes=('', '_global'))
        .merge(df_past_7_days_sales, on=['seller_sku', 'seller_id'], how='left', suffixes=('', '_global'))
    )
    logger.info("Dataframes merged successfully")

    # Clear and insert into BigQuery
    dataset_table_id = settings.TABLE_PREDICTED_SALES
    logger.info(f"Clearing data from {dataset_table_id} for today's date")
    bigquery.run_query(f"DELETE FROM {dataset_table_id} WHERE DATE(insertion_date) = CURRENT_DATE()")

    logger.info("Inserting new data into BigQuery")
    bigquery.insert_dataframe(df_consolidado, dataset_table_id)
    logger.info("Data inserted successfully")

    return ("Concurrents predicted sales information processed successfully.", 200)


def get_sales_table(table_id, bigquery):
    query = f'''
    SELECT 
        date_created, 
        seller_sku, 
        seller_id, 
        context_channel, 
        quantity 
    FROM {table_id}
    WHERE payment_status = 'approved'
    '''
    df = bigquery.run_query(query)
    # Ensure datetime index
    df['date_created'] = pd.to_datetime(df['date_created'])
    df.set_index('date_created', inplace=True)
    return df


def create_max_history_per_sku(df_sales, context_channel=True):
    """
    Compute the max rolling 7-day and 21-day sums and their corresponding dates for each SKU.
    If context_channel=True, compute per SKU+seller_id+context_channel.
    If context_channel=False, compute per SKU+seller_id only (global).
    """

    group_cols = ['seller_sku', 'seller_id']
    if context_channel:
        group_cols.append('context_channel')

    # Sort index to ensure proper resampling
    df_sales = df_sales.sort_index()

    # Resample daily sums
    # This creates a DataFrame with daily frequency rows and sum of quantities per day.
    df_daily = (
        df_sales
        .groupby(group_cols)
        .resample('D')['quantity']
        .sum()
        .reset_index()
        .sort_values(group_cols + ['date_created'])
    )

    # Set multi-index for rolling operations
    df_daily.set_index(group_cols + ['date_created'], inplace=True)

    # Compute rolling sums
    rolling_7 = df_daily.groupby(group_cols)['quantity'].rolling(7, min_periods=1).sum()
    rolling_21 = df_daily.groupby(group_cols)['quantity'].rolling(21, min_periods=1).sum()

    # Compute max values
    max_7 = rolling_7.groupby(group_cols).max().reset_index(name='max_sold_7_days_history')
    max_21 = rolling_21.groupby(group_cols).max().reset_index(name='max_sold_21_days_history')

    # Identify the dates for the max values
    # idxmax returns the index labels at the max value
    idxmax_7 = rolling_7.groupby(group_cols).idxmax().reset_index(name='idx_7')
    idxmax_21 = rolling_21.groupby(group_cols).idxmax().reset_index(name='idx_21')

    # Merge idxmax results back with df_daily to get the actual date of max
    # idxmax_7['idx_7'] and idxmax_21['idx_21'] will contain a tuple (seller_sku, seller_id, [context_channel], date)
    # Extract the date from this tuple
    idxmax_7['date_end_max_sold_history_7_days'] = idxmax_7['idx_7'].apply(lambda x: x[-1])
    idxmax_21['date_end_max_sold_history_21_days'] = idxmax_21['idx_21'].apply(lambda x: x[-1])

    # Merge everything together
    df_result = (
        max_7
        .merge(idxmax_7[group_cols + ['date_end_max_sold_history_7_days']], on=group_cols)
        .merge(max_21, on=group_cols)
        .merge(idxmax_21[group_cols + ['date_end_max_sold_history_21_days']], on=group_cols)
    )

    df_result['insertion_date'] = datetime.now()

    return df_result


def create_past_sales(df_sales, num_days):
    logger.info(f"Computing past {num_days} days sales")

    # Set the timezone for São Paulo (UTC-3 or UTC-2 depending on DST)
    sao_paulo = pytz.timezone('America/Sao_Paulo')

    # Get yesterday's date in São Paulo timezone and set to 23:59:59
    end_date = datetime.now(sao_paulo).replace(hour=23, minute=59, second=59) - timedelta(days=1)
    start_date = end_date - timedelta(days=num_days - 1)
    start_date = start_date.replace(hour=0, minute=0, second=0)

    # Convert to UTC for comparison
    start_date_utc = start_date.astimezone(pytz.utc)
    end_date_utc = end_date.astimezone(pytz.utc)

    # Filter the DataFrame for the given date range
    mask = (df_sales.index >= start_date_utc) & (df_sales.index <= end_date_utc)
    df_filtered_sales = df_sales[mask]

    logger.info(f"Filtered {len(df_filtered_sales)} rows for date range: {start_date_utc} to {end_date_utc}")

    # Group by 'seller_sku', 'seller_id', and 'context_channel' (with channel)
    df_past_sales_with_channel = (
        df_filtered_sales
        .groupby(['seller_sku', 'seller_id', 'context_channel'], as_index=False)
        .agg({'quantity': 'sum'})
        .rename(columns={'quantity': f'past_{num_days}_sales'})
    )

    logger.info(f"Computed past {num_days} days sales with channel")

    df_filtered_sales.fillna(0, inplace=True)
    # Group by 'seller_sku' and 'seller_id' (global, without channel)
    df_past_sales_global = (
        df_filtered_sales
        .groupby(['seller_sku', 'seller_id'], as_index=False)
        .agg({'quantity': 'sum'})
        .rename(columns={'quantity': f'past_{num_days}_sales_global'})
    )

    logger.info(f"Computed past {num_days} days sales globally (without channel)")

    logger.info(f"Merged past {num_days} days sales with global sales")

    logger.debug(f"Start date: {start_date}, End date: {end_date}, Rows: {len(df_past_sales_global)}")
    
    return df_past_sales_with_channel, df_past_sales_global

In [34]:
df_past_sales_ = get_max_sales_history({})

2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,909 __main__ [INFO] Starting the get_max_sales_history process
2024-12-18 18:23:22,927 __main__ [INFO] BigQueryManager initialized
2024-12-18 18:23:22

Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,443 __main__ [INFO] Loaded 316954 sales records from BigQuery
2024-12-18 18:23:25,462 __main__ [INFO] Computing maximum sales history per SKU and channel
2024-1

Data inserted into datalake-v2-424516.predictions.sales_v2.


In [ ]:
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
logger.info("BigQueryManager initialized")
# Get sales data
df_sales = get_sales_table(settings.TABLE_ORDERS, bigquery)

num_days = 21

sao_paulo = pytz.timezone('America/Sao_Paulo')
# Get yesterday's date in São Paulo timezone and set to 23:59:59
end_date = datetime.now(sao_paulo).replace(hour=23, minute=59, second=59) - timedelta(days=1)
start_date = end_date - timedelta(days=num_days - 1)
start_date = start_date.replace(hour=0, minute=0, second=0)
# Convert to UTC for comparison
start_date_utc = start_date.astimezone(pytz.utc)
end_date_utc = end_date.astimezone(pytz.utc)
# Filter the DataFrame for the given date range
mask = (df_sales.index >= start_date_utc) & (df_sales.index <= end_date_utc)
df_filtered_sales = df_sales[mask]
logger.info(f"Filtered {len(df_filtered_sales)} rows for date range: {start_date_utc} to {end_date_utc}")
# Group by 'seller_sku', 'seller_id', and 'context_channel' (with channel)
df_past_sales_with_channel = (
    df_filtered_sales
    .groupby(['seller_sku', 'seller_id', 'context_channel'], as_index=False)
    .agg({'quantity': 'sum'})
    .rename(columns={'quantity': f'past_{num_days}_sales'})
)

logger.info(f"Computed past {num_days} days sales with channel")
df_past_sales_with_channel.fillna(0, inplace=True)
# Group by 'seller_sku' and 'seller_id' (global, without channel)
df_past_sales_global = (
    df_filtered_sales
    .groupby(['seller_sku', 'seller_id'], as_index=False)
    .agg({'quantity': 'sum'})
    .rename(columns={'quantity': f'past_{num_days}_sales_global'})
)

df_merged = pd.merge(
     df_past_sales_with_channel,
     df_past_sales_global,
     on=['seller_sku', 'seller_id'],
     how='left'
 )

2024-12-18 18:10:15,927 __main__ [INFO] BigQueryManager initialized
2024-12-18 18:10:15,927 __main__ [INFO] BigQueryManager initialized
2024-12-18 18:10:15,927 __main__ [INFO] BigQueryManager initialized
2024-12-18 18:10:15,927 __main__ [INFO] BigQueryManager initialized
2024-12-18 18:10:15,927 __main__ [INFO] BigQueryManager initialized
2024-12-18 18:10:15,927 __main__ [INFO] BigQueryManager initialized


Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


2024-12-18 18:10:18,304 __main__ [INFO] Filtered 25154 rows for date range: 2024-11-27 03:00:00.291515+00:00 to 2024-12-18 02:59:59.291515+00:00
2024-12-18 18:10:18,304 __main__ [INFO] Filtered 25154 rows for date range: 2024-11-27 03:00:00.291515+00:00 to 2024-12-18 02:59:59.291515+00:00
2024-12-18 18:10:18,304 __main__ [INFO] Filtered 25154 rows for date range: 2024-11-27 03:00:00.291515+00:00 to 2024-12-18 02:59:59.291515+00:00
2024-12-18 18:10:18,304 __main__ [INFO] Filtered 25154 rows for date range: 2024-11-27 03:00:00.291515+00:00 to 2024-12-18 02:59:59.291515+00:00
2024-12-18 18:10:18,304 __main__ [INFO] Filtered 25154 rows for date range: 2024-11-27 03:00:00.291515+00:00 to 2024-12-18 02:59:59.291515+00:00
2024-12-18 18:10:18,304 __main__ [INFO] Filtered 25154 rows for date range: 2024-11-27 03:00:00.291515+00:00 to 2024-12-18 02:59:59.291515+00:00
2024-12-18 18:10:18,306 __main__ [INFO] Computed past 21 days sales with channel
2024-12-18 18:10:18,306 __main__ [INFO] Computed 

In [23]:
df_past_sales_with_channel

,seller_sku,seller_id,context_channel,past_21_sales
0,000001,569119547,marketplace,2
1,000999,569119547,marketplace,5
2,001002,569119547,marketplace,1
3,001210,569119547,marketplace,31
4,001210,569119547,mshops,2
...,...,...,...,...
1461,ZTS-US8-B,189643563,mshops,13
1462,ZTS-US8-W,189643563,marketplace,4
1463,ZTS-US8-W,189643563,mshops,21
1464,ZVL-PRO,189643563,marketplace,21


In [32]:
df_past_sales_[df_past_sales_['seller_sku'] == 'EKXZ-310']

,seller_sku,seller_id,context_channel,max_sold_7_days_history,date_end_max_sold_history_7_days,max_sold_21_days_history,date_end_max_sold_history_21_days,insertion_date,past_21_sales,past_7_sales,max_sold_7_days_history_global,date_end_max_sold_history_7_days_global,max_sold_21_days_history_global,date_end_max_sold_history_21_days_global,insertion_date_global,past_21_sales_global,past_7_sales_global
4162,EKXZ-310,189643563,marketplace,118.0,2024-11-15 00:00:00+00:00,237.0,2024-11-25 00:00:00+00:00,2024-12-18 18:22:38.744212,4,1,118.0,2024-11-15 00:00:00+00:00,237.0,2024-11-25 00:00:00+00:00,2024-12-18 18:22:46.563153,4,1
4163,EKXZ-310,189643563,mshops,1.0,2024-05-29 00:00:00+00:00,1.0,2024-05-29 00:00:00+00:00,2024-12-18 18:22:38.744212,<NA>,<NA>,118.0,2024-11-15 00:00:00+00:00,237.0,2024-11-25 00:00:00+00:00,2024-12-18 18:22:46.563153,4,1
